In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import librosa

#to save trained models
import joblib
import pickle
import json

#EEG Module


In [ ]:
#Libraries helping with file retrival
import os
from os import listdir
from os.path import isfile, join

#Visualization and data manipulation libraries
import matplotlib.pyplot as plt
import scipy.io
from scipy.signal import butter, lfilter

import pandas as pd
import numpy as np

import seaborn as sns
sns.set(font_scale=1.2)

#Model development and understanding
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn  import metrics


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('/content/drive/My Drive/audio_data/eeg_df.csv')

In [ ]:
df.head()

,patient_identifier,lf_alpha_fear_E36,lf_beta_fear_E36,lf_delta_fear_E36,lf_theta_fear_E36,lf_mean_fear_E36,lf_max_fear_E36,lf_min_fear_E36,lf_median_fear_E36,lf_alpha_happy_E36,...,lf_fear_min,lf_fear_max,lf_sad_mean,lf_sad_median,lf_sad_min,lf_sad_max,lf_resting_mean,lf_resting_median,lf_resting_min,lf_resting_max
0,2010008,0.003473,0.003368,0.015247,0.007565,0.059222,517.229278,0.000041,0.003682,0.002941,...,0.000031,723.153946,0.072019,0.002149,0.000171,629.557370,108281.564986,6288.040454,1128.528637,9.583322e+08
1,2010011,0.004254,0.002348,0.016904,0.006377,0.089253,867.522513,0.000025,0.002415,0.002548,...,0.000150,953.367554,0.107683,0.004103,0.000072,984.899036,85553.767649,4110.453724,45.625631,8.476602e+08
2,2010015,0.001797,0.001225,0.005069,0.002325,0.011508,60.477420,0.000012,0.001242,0.002351,...,0.000040,249.871548,0.048740,0.004281,0.000191,286.843937,81978.488000,5941.513906,823.625097,5.838479e+08
3,2010013,0.002016,0.001013,0.003829,0.002445,0.024019,241.903701,0.000014,0.001012,0.003692,...,0.000024,468.864574,0.049950,0.002874,0.000037,462.087615,54210.252136,1901.610336,14.034971,5.524028e+08
4,2010012,0.009081,0.004736,0.040018,0.015150,0.038334,180.524548,0.000026,0.005003,0.003708,...,0.000102,533.492279,0.054746,0.003305,0.000029,497.015771,91590.362795,9364.358228,570.766807,4.573432e+08


In [ ]:
#dividing the data set into test and train. Here we have to create a stratified
#sample to create the train and test datasets.
sample = df.groupby('type', group_keys=False).apply(lambda x: x.sample(round(30*0.7),random_state=42))
left_over = df[~(df.index.isin(list(sample.index)))]

In [ ]:
# eeg_df = df.loc[df['patient_identifier'] == 2010013]

In [ ]:
# eeg_df.head()

In [ ]:
# print(eeg_df['type'])

0    1
Name: type, dtype: int64


In [ ]:
#variables for test and train
y = sample['type']
X = sample.drop(columns = 'type')
X_test = left_over.drop(columns = 'type')
y_test = left_over['type']

In [ ]:
np.save('/content/drive/My Drive/audio_data/eeg_xtrain.npy', X)

In [ ]:

scalar = StandardScaler()
X_scale = scalar.fit(X).transform(X)
X_scale_test = scalar.transform(X_test)

In [ ]:
# #variables for test and train
# y = eeg_df['type']
# X = eeg_df.drop(columns = 'type')

In [ ]:
# #Before building the PCA, we will first scale the dataset to ensure that
# #there are no differences in the possible range of values
# scalar = StandardScaler()
# eeg_df = scalar.fit(X).transform(X)

In [ ]:
def classifier(X,y,X_test,y_test):
  """
  Input:

  X - Training set
  y - Training outcome
  X_test - Test set
  y_test - Test outcome
  
  Output :
  total - dictionary summarizing performances of various models

  """
  total = dict()
  # rf = RandomForestClassifier(random_state=42)
  # lr = LogisticRegression(random_state=42)
  # knn = KNeighborsClassifier()
  # svc = SVC(random_state=42)
  # xgb = XGB_Classifier
  # lgbm = LGB_Classifier
  bg = BaggingClassifier()
  # dt = DecisionTreeClassifier()
  # nn = model1
  # lda1 = LinearDiscriminantAnalysis()
  model_array = [bg]
  model_vals = ['bagging classifier']
  i = 0
  for clf in model_array:
    result = dict()
    kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    cv_results = cross_val_score(clf, X, y, cv=kfold, scoring='accuracy')
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    result['accuracy']=round(accuracy_score(y_test, y_pred),2)
    result['precision']=round(precision_score(y_test, y_pred),2)
    result['recall']=round(recall_score(y_test, y_pred),2)

    print("predictions: ", y_pred)
    print("classifier: ", clf)
    print(classification_report(y_test, y_pred))
    test_acc_e = metrics.accuracy_score(y_test, y_pred)
    # cm = confusion_matrix(y_test, y_pred)
    # ax = sns.heatmap(cm, linecolor='white', cmap='PiYG', linewidth=1, annot=True, fmt='')

    joblib.dump(bg, '/content/drive/My Drive/saved_models/eeg_model3.pkl')

    total[model_vals[i]]=result
    i = i+1
    
  return total, test_acc_e

In [ ]:
Vals = list(X.columns)

In [ ]:
for emotion in ['fear','sad','happy','resting','erp','all']:
  for type_vars in ['nl','lf','all']:  

    if( emotion == 'sad' and type_vars == 'lf'):
      print("emotion: ", emotion)
      print("features: ", type_vars)

      if emotion =='erp':
        indices_to_keep = [i for i in range(len(Vals)) if type_vars in Vals[i] and 'resting' not in Vals[i]]
        if type_vars =='all':
          indices_to_keep = [i for i in range(len(Vals)) if 'resting' not in Vals[i]]
      elif emotion =='all':
        indices_to_keep = [i for i in range(len(Vals)) if type_vars in Vals[i]]
        if type_vars =='all':
          indices_to_keep = [i for i in range(len(Vals))]
      else:
        indices_to_keep = [i for i in range(len(Vals)) if type_vars in Vals[i] and emotion in Vals[i]]
        if type_vars =='all':
          indices_to_keep = [i for i in range(len(Vals)) if emotion in Vals[i]]
        
      temp, test_acc_e = classifier(X_scale[:,indices_to_keep],y,X_scale_test[:,indices_to_keep],y_test)
      temp['base'] = type_vars
      temp['emotion'] = emotion
      # final = pd.DataFrame(temp).reset_index()
      print(temp)


emotion:  sad
features:  lf
predictions:  [0 1 0 0 1 0 1 0 0 0 1]
classifier:  BaggingClassifier()
              precision    recall  f1-score   support

           0       0.86      0.75      0.80         8
           1       0.50      0.67      0.57         3

    accuracy                           0.73        11
   macro avg       0.68      0.71      0.69        11
weighted avg       0.76      0.73      0.74        11

{'bagging classifier': {'accuracy': 0.73, 'precision': 0.5, 'recall': 0.67}, 'base': 'lf', 'emotion': 'sad'}


In [ ]:
###
print(indices_to_keep)

[17, 18, 19, 20, 21, 22, 23, 24, 50, 51, 52, 53, 54, 55, 56, 57, 83, 84, 85, 86, 87, 88, 89, 90, 116, 117, 118, 119, 120, 121, 122, 123, 149, 150, 151, 152, 153, 154, 155, 156, 182, 183, 184, 185, 186, 187, 188, 189, 215, 216, 217, 218, 219, 220, 221, 222, 248, 249, 250, 251, 252, 253, 254, 255, 281, 282, 283, 284, 285, 286, 287, 288, 314, 315, 316, 317, 318, 319, 320, 321, 347, 348, 349, 350, 351, 352, 353, 354, 380, 381, 382, 383, 384, 385, 386, 387, 413, 414, 415, 416, 417, 418, 419, 420, 446, 447, 448, 449, 450, 451, 452, 453, 479, 480, 481, 482, 483, 484, 485, 486, 512, 513, 514, 515, 516, 517, 518, 519, 721, 722, 723, 724, 737, 738, 739, 740]


In [ ]:
with open("/content/drive/My Drive/saved_models/indices_to_keep.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(indices_to_keep, f) 

# with open("file.json", 'r') as f:
#     score = json.load(f)



In [ ]:
print(test_acc_e.round(2))

0.91


In [ ]:
# Save the model as a pickle in a file
# joblib.dump(bg, '/content/drive/My Drive/saved_models/eeg_model.pkl')

# Audio Module

---



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#Keras
import keras
from keras import models
from keras import layers

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import metrics
from sklearn.metrics import classification_report

In [ ]:
# df = pd.read_csv('/content/drive/My Drive/audio_data/combined_df.csv', header=None)

In [ ]:
X_train = np.load('/content/drive/My Drive/audio_data/xtrain.npy')
X_test = np.load('/content/drive/My Drive/audio_data/xtest.npy')
y_train = np.load('/content/drive/My Drive/audio_data/ytrain.npy')
y_test = np.load('/content/drive/My Drive/audio_data/ytest.npy')


In [ ]:
#normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# creating a model
model = models.Sequential()
model.add(layers.Dense(128, activation='relu' ))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=32)

Epoch 1/100
16/16 [==============================] - 3s 3ms/step - loss: 0.6552 - accuracy: 0.6172
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6127 - accuracy: 0.6797
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5843 - accuracy: 0.6816
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5577 - accuracy: 0.6875
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5330 - accuracy: 0.7285
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5079 - accuracy: 0.7520
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4768 - accuracy: 0.7773
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.4463 - accuracy: 0.7910
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.4238 - accuracy: 0.8047
Epoch 10/100
16/16 [==============================] - 0s 3ms/step - loss: 0.3845 - accuracy: 0.8496
Epoch 11/

In [ ]:
# calculate accuracy
test_loss_a, test_acc_a = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc_a)




4/4 [==============================] - 0s 3ms/step - loss: 0.8847 - accuracy: 0.8281
test_acc:  0.828125


In [ ]:
# predictions
predictions = model.predict(X_test)
# print(predictions)

In [ ]:
print(classification_report(y_test, predictions.round()))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89        96
           1       0.67      0.62      0.65        32

    accuracy                           0.83       128
   macro avg       0.77      0.76      0.77       128
weighted avg       0.82      0.83      0.83       128



In [ ]:
########################## To save the model #############################################

In [ ]:
#save model
model.save('/content/drive/My Drive/saved_models/audio_model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/saved_models/audio_model/assets


In [ ]:
#load model
# new_model = tf.keras.models.load_model('saved_model/my_model')

#Video Module

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
#Video

# print("Video : ")

from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

def processData(data):
    # print(data)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))

    X = np.mean(X, 0)
    X = np.array(X.reshape(1, -1))
    # print(X)
    # print(X.shape)
    return X

def getData(filename):
    # print(filename)
    data = pd.read_csv(filename,delimiter=',')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    # X5 = getData(file5)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    # print(X)
    # print(X.shape)
    return X

count0 = 0
count1 = 0

def makeDataset(location, folder):
    global count0
    global count1
    file  = np.array(pd.read_csv(location,delimiter=',',encoding='utf-8', low_memory=False))[:, 0:2]
    X_temp = np.zeros((1,388))
    Y_temp = []
    for i in range(len(file)):
      if(int(file[i][1]) == 0):
        count0 += 1
        if(count0 >39):
          continue
      patientID = (str(int(file[i][0])))
      #print(patientID)
      string = '/content/drive/My Drive/MCA Dataset/' + folder +"/"+ patientID
      X_temp = np.concatenate((X_temp, makeDataPoint(string)), 0)
      Y_temp.append(int(file[i][1]))
    X_temp = np.delete(X_temp,0,0)
    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp





In [ ]:
print("train")
X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')
print("dev")
X_dev, Y_dev = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')
count0 = 0
X_train = np.concatenate((X_train, X_dev), 0  )
Y_train = np.concatenate((Y_train, Y_dev), 0  )

print("test")
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')

print(Y_train)

train
dev


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,10

test
[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]


In [ ]:
# np.save('/content/drive/My Drive/audio_data/videoML_xtrain.npy', X_train)
# np.save('/content/drive/My Drive/audio_data/videoML_xtest.npy', X_test)
# np.save('/content/drive/My Drive/audio_data/videoML_ytrain.npy', Y_train)
# np.save('/content/drive/My Drive/audio_data/videoML_ytest.npy', Y_test)

In [ ]:
# X_train = np.load('/content/drive/My Drive/audio_data/videoML_xtrain.npy')
# X_test = np.load('/content/drive/My Drive/audio_data/videoML_xtest.npy')
# Y_train = np.load('/content/drive/My Drive/audio_data/videoML_ytrain.npy')
# Y_test = np.load('/content/drive/My Drive/audio_data/videoML_ytest.npy')

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)
print(X_test)

[[ 0.10597545  0.7562539   0.49019156 ... -1.79536994  0.17531763
  -0.31491738]
 [-0.52555507 -0.90887952  0.6385864  ... -1.42678952  0.28582234
  -0.90343702]
 [ 2.45928632  0.60050928  0.4057917  ... -1.35444052  1.22340489
  -0.31916397]
 ...
 [ 1.24938957 -0.04655008  3.69939457 ... -0.14465841  0.14506043
   0.4184208 ]
 [ 1.50997096 -0.56469359  0.81869417 ... -0.28571969  1.22988828
  -0.69636632]
 [ 2.0434696  -0.85899382 -0.46430977 ...  1.43894123 -1.47238811
  -1.89642009]]
[[-0.99737441  0.60996015  0.28578804 ... -0.07797502  0.58869598
  -2.23244013]
 [-0.42282901  0.77070598 -1.73254779 ...  0.18290487 -1.04139004
  -0.54808756]
 [-0.31115127  0.72214897 -0.28977623 ... -1.5734965  -1.34253206
  -1.08963453]
 ...
 [ 0.00739774  0.60056331  0.79091778 ...  1.52879386  0.00398596
  -1.81896955]
 [-0.28647955 -2.26268319 -0.67844973 ... -0.42977991  2.74780748
  -2.38416019]
 [ 0.5345647   0.19975499  1.5999336  ... -0.89852531 -0.02213174
  -1.01777375]]


In [ ]:
k = 45
#Training and Predicting  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,Y_train)
pred_knn = neigh.predict(X_train)
pred_test_knn = neigh.predict(X_test)
pred_test_knn[0:5]

Ks = 50
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,Y_train)
    pred_test_knn=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(Y_test, pred_test_knn)

    
    std_acc[n-1]=np.std(pred_test_knn==Y_test)/np.sqrt(pred_test_knn.shape[0])

mean_acc

print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

The best accuracy was with 0.6595744680851063 with k= 2


In [ ]:
neigh = KNeighborsClassifier(n_neighbors = 2).fit(X_train,Y_train)
pred_test_knn=neigh.predict(X_test)

In [ ]:
print(metrics.classification_report(Y_test, pred_test_knn))

              precision    recall  f1-score   support

           0       0.77      0.73      0.75        33
           1       0.44      0.50      0.47        14

    accuracy                           0.66        47
   macro avg       0.61      0.61      0.61        47
weighted avg       0.67      0.66      0.67        47



In [ ]:
test_acc_v = metrics.accuracy_score(Y_test, pred_test_knn)
print(test_acc_v.round(2))

0.66


In [ ]:
########################## To save the model #############################################

In [ ]:
# Save the model as a pickle in a file
joblib.dump(neigh, '/content/drive/My Drive/saved_models/video_model.pkl')
 
# # Load the model from the file
# knn_from_joblib = joblib.load('filename.pkl')
 
# # Use the loaded model to make predictions
# knn_from_joblib.predict(X_test)

['/content/drive/My Drive/saved_models/video_model.pkl']

# Fusion for Depression Detection (0 | 1)

In [ ]:
#load models
eeg_model = joblib.load('/content/drive/My Drive/saved_models/eeg_model.pkl')
video_model = joblib.load('/content/drive/My Drive/saved_models/video_model.pkl')
# audio_model = joblib.load('/content/drive/My Drive/saved_models/audio_knn.pkl')
audio_model = tf.keras.models.load_model('/content/drive/My Drive/saved_models/audio_model')

## Get data instances

### audio data

In [ ]:
audio_1 = pd.DataFrame([[-0.2596735, -0.50194013, -0.46613182, -0.5643582, -0.47627463, -0.46204195, -0.40727931, 0.74268959, -1.7024481, 0.28516535, -0.85800545, -1.18019641, -1.05231356, -2.73309019, 1.14803668, -1.3624679, -0.87839032, 1.85343487, -2.07122509, -0.61635689, 1.03520932, -1.1271249, -1.83945593, -0.24893407, 0.46151611, -1.82637981]])

In [ ]:
audio_2 = pd.DataFrame([[0.569320023, 0.017553795, 1179.037573, 1526.276326, 2431.840489, 0.04723667, -378.723999, 89.88813019, 34.43695068, 23.43453407, 9.176048279, 14.36908436, 2.037665129, -0.586535037, 3.183698654, -2.697319269, 3.535515547, 3.85468936, -0.833142698, 2.360462666, 3.653418779, 2.640903234, 2.180514812, -0.018858721,	2.898318291, 0.768064797, 1]])

In [ ]:
audio_df = audio_1

In [ ]:
audio_df.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,-0.259674,-0.50194,-0.466132,-0.564358,-0.476275,-0.462042,-0.407279,0.74269,-1.702448,0.285165,...,-0.87839,1.853435,-2.071225,-0.616357,1.035209,-1.127125,-1.839456,-0.248934,0.461516,-1.82638


### Video data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

def processData(data):
    # print(data)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))

    X = np.mean(X, 0)
    X = np.array(X.reshape(1, -1))
    # print(X)
    # print(X.shape)
    return X

def getData(filename):
    # print(filename)
    data = pd.read_csv(filename,delimiter=',')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    # X5 = getData(file5)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    # print(X)
    # print(X.shape)
    return X

count0 = 0
count1 = 0

def makeDataset(location, folder):
    global count0
    global count1
    file  = np.array(pd.read_csv(location,delimiter=',',encoding='utf-8', low_memory=False))[:, 0:2]
    X_temp = np.zeros((1,388))
    Y_temp = []
    for i in range(len(file)):
      if(int(file[i][1]) == 0):
        count0 += 1
        if(count0 >39):
          continue
      patientID = (str(int(file[i][0])))
      #print(patientID)
      ############## patientID check ###################################################################421
      if(patientID == '421'):
      ################################################################################  
        string = '/content/drive/My Drive/MCA Dataset/' + folder +"/"+ patientID
        X_temp = np.concatenate((X_temp, makeDataPoint(string)), 0)
        Y_temp.append(int(file[i][1]))
    X_temp = np.delete(X_temp,0,0)
    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp

In [ ]:
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')

In [ ]:
X_train = np.load('/content/drive/My Drive/audio_data/videoML_xtrain.npy', allow_pickle=True)

In [ ]:
print(Y_test)

[1]


In [ ]:
video_df = X_test

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
video_df = scaler.transform(X_test)

In [ ]:
# print(video_df)

### EEG Data

In [ ]:
df = pd.read_csv('/content/drive/My Drive/audio_data/eeg_df.csv')
#2010008  #2020029

In [ ]:
eeg_df = df.loc[df['patient_identifier'] == 2010008]

In [ ]:
eeg_df.head()

,patient_identifier,lf_alpha_fear_E36,lf_beta_fear_E36,lf_delta_fear_E36,lf_theta_fear_E36,lf_mean_fear_E36,lf_max_fear_E36,lf_min_fear_E36,lf_median_fear_E36,lf_alpha_happy_E36,...,lf_fear_min,lf_fear_max,lf_sad_mean,lf_sad_median,lf_sad_min,lf_sad_max,lf_resting_mean,lf_resting_median,lf_resting_min,lf_resting_max
0,2010008,0.003473,0.003368,0.015247,0.007565,0.059222,517.229278,0.000041,0.003682,0.002941,...,0.000031,723.153946,0.072019,0.002149,0.000171,629.55737,108281.564986,6288.040454,1128.528637,9.583322e+08


In [ ]:
print(eeg_df['type'])

0    1
Name: type, dtype: int64


In [ ]:
eeg_X_train = np.load('/content/drive/My Drive/audio_data/eeg_xtrain.npy')


In [ ]:
scalar = StandardScaler()
eeg_X_train = scalar.fit_transform(eeg_X_train)
eeg_df = scalar.transform(eeg_df.drop(columns = 'type'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
print(eeg_df)

[[ 1.61563819e+00 -4.50894713e-01 -4.60140967e-01 -4.56727841e-01
  -4.58313071e-01 -4.10939620e-01 -3.87063717e-01 -3.04562619e-01
  -4.84490727e-01 -3.26429852e-01 -2.82589942e-01 -3.20362385e-01
  -3.25881864e-01 -3.21790877e-01 -3.14155764e-01 -2.78849050e-01
  -2.31495806e-01 -3.15056489e-01 -2.91984225e-01 -3.15082806e-01
  -3.11006815e-01 -3.57021763e-01 -3.73870920e-01 -3.15851054e-01
  -2.66286122e-01 -3.94232546e-01 -2.53590613e-01  2.39326147e+00
  -2.10358706e-01 -6.54344760e-01  2.66331747e+00 -3.23366705e-01
  -7.69734460e-01  2.81577246e+00 -7.02643332e-01 -7.73557838e-01
  -1.06787995e+00 -9.81897280e-01 -5.57356105e-01 -4.35917411e-01
  -3.45091381e-01 -8.99188337e-01 -1.37939763e-01 -1.94068637e-01
  -3.38828700e-01 -3.54616060e-01 -3.72831111e-01 -3.29243308e-01
  -2.03981784e-01 -5.20414160e-02 -2.60978423e-01 -2.11474405e-01
  -4.12473953e-01 -3.87287520e-01 -4.36664484e-01 -4.26630100e-01
  -1.45448812e-01 -1.51939289e-01 -5.86184782e-01 -7.02900056e-02
   1.69604

In [ ]:
with open("/content/drive/My Drive/saved_models/indices_to_keep.json", 'r') as f:
    indices_to_keep = json.load(f)



## Fusion

In [ ]:
audio_pred = audio_model.predict(audio_df)
print(audio_pred)
print(int(audio_pred.round()))

[[0.99999106]]
1


In [ ]:
video_pred = video_model.predict(video_df)
print(video_pred)

[1]


In [ ]:
eeg_pred = eeg_model.predict(eeg_df[:,indices_to_keep])
print(eeg_pred)

[1]


In [ ]:
if(eeg_pred == 1 or int(audio_pred.round()) == 1 or video_pred == 1):
  print("1: Depressed")
else:
  print("0: Not Depressed")

1: Depressed


#Fusion for Depression Sevierty calculation

In [ ]:
# print(test_acc_e,test_acc_a,test_acc_v )

In [ ]:
#EEG testing accuracy contribution in multimodal system in percent
Y_E = 100 * test_acc_e / (test_acc_a + test_acc_e + test_acc_v)
print(Y_E.round(2))

In [ ]:
#Audio module testing accuracy contribution in multimodal system in percent
Y_A = 100 * test_acc_a / (test_acc_a + test_acc_e + test_acc_v)
print(Y_A.round(2))

In [ ]:
#Video module testing accuracy contribution in multimodal system in percent
Y_V = 100 * test_acc_v / (test_acc_a + test_acc_e + test_acc_v)
print(Y_V.round(2))

While calulating complete accuracy (severity of depression by considering all three factors of patient or user ) 

Accuracy of test sample in percent

In [ ]:
test_accuracy = Y_E * test_acc_e + Y_A * test_acc_a + Y_V * test_acc_v
print(test_accuracy.round(2))

# --------------------------------------------------------------------------------------------------

In [ ]:
audio_path = '/content/drive/My Drive/audio_data/interim/P462/P462_no_silence.wav'

###############################################
#load the audio file
x , sr = librosa.load(audio_path)

chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)

#indicates loudness(root mean square energy)
rmse = librosa.feature.rms(x)

spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
           
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))

#number of times a signal croses the horizontal axis
zero_crossings = librosa.feature.zero_crossing_rate(x)

spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)
              
spectral_rolloff = librosa.feature.spectral_rolloff(x, sr=sr)
              
mfccs = librosa.feature.mfcc(x, sr=sr)
print(mfccs.shape)
             

#append all features together in one file
to_append = f' {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spectral_centroids)} {np.mean(spec_bw)} {np.mean(spectral_rolloff)} {np.mean(zero_crossings)}'

for e in mfccs:
  to_append += f' {np.mean(e)}'

###################################################

to_append2 =  to_append.split()
to_append3 = np.array(to_append2)

###############################################
X_train_audio = np.load('/content/drive/My Drive/audio_data/xtrain.npy')

scaler = StandardScaler()
X_train_audio = scaler.fit_transform(X_train_audio)
audio_df = scaler.transform(to_append3.reshape(1, -1))

(20, 55639)
